In [ ]:
from bs4 import BeautifulSoup

xml_file_path = 'VCORE_VALEO_STD_VTR.html'

with open(xml_file_path, 'r', encoding='utf-8') as file:
    xml_content = file.read()

soup = BeautifulSoup(xml_content, "html.parser")
print(soup.prettify())

In [ ]:
import pandas as pd

blocks = soup.find_all('div', class_='Indentation')
resulted_block = []

for block in blocks:
    feature_tag = block.find('td', text='Feature: ')
    requirements_tag = block.find('td', text='Tested requirements: ')
    table = block.find('table', class_='ResultTable')

    if feature_tag and requirements_tag and table:
        feature = feature_tag.find_next('td').text.strip()
        requirements = requirements_tag.find_next('td').text.strip()

        data = []
        for row in table.find_all('tr')[1:]:
            cells = row.find_all('td')
            row_data = [requirements, feature] + [cell.text.strip() for cell in cells]
            data.append(row_data)

        df = pd.DataFrame(data)
        filtered_df = df[df.iloc[:, 3].str.contains('step', case=False, na=False)]
        filtered_df = filtered_df.drop(filtered_df.columns[2], axis=1).iloc[:, :5]
        resulted_block.append(filtered_df)

# Concatenate all DataFrames into a single DataFrame
result_df = pd.concat(resulted_block, ignore_index=True)

# Display the DataFrame
print(result_df)

# Save the DataFrame to an Excel file
result_df.to_excel('htmlParser.xlsx', index=False)

In [ ]:
df = pd.read_excel('htmlParser.xlsx')
# Save the DataFrame to a CSV file
df.to_csv('htmlParser.csv', index=False)
#The end